In [1]:
from math import log2
import essentia.standard
import numpy as np
from collections import Counter
import os

<h1>=====================================</h1>

In [2]:
def read_melody_files(filename):
    with open(filename) as f:
        _notes = f.read()

    _notes = _notes.split(',')
    return [float(i) for i in _notes]

In [3]:
def to_cents(song):
    cents = []
    for i, f in enumerate(song):
        if song[i] > 0:
            cents.append(1200 * log2(song[i]/55))
        else:
            cents.append(0)
    return np.array(cents)

In [4]:
def to_semitones(cents):
    semitones=[]
    for i, f in enumerate(cents):
        semitones.append(cents[i]//100)
    return np.array(semitones)

In [5]:
def map_into_single_octave(semitones):
    mapped=[]
    min_n=1
    max_n=12
    min_d = np.min(semitones[np.nonzero(semitones)])
    max_d= max(semitones)
    for i, f in enumerate(semitones):
        if semitones[i] > 0:
            mapped.append(((semitones[i]-min_d)*(max_n-min_n))//(max_d-min_d)+min_n)
        else:
            mapped.append(0)
    return np.array(mapped)

In [6]:
def get_histogram(pitch_class):
    limitator=0
    histogram=[]
    while limitator < len(pitch_class):
        frame=[]
        ini=limitator
        fin=ini+49
        if(fin>len(pitch_class)-1):
            fin=len(pitch_class)-1
        counter=Counter(pitch_class[ini:fin])
        for i in range(12):
            frame.append(counter[i+1])
        for x in range(len(frame)):
            if frame[x] == 0:
                continue
            num = frame[x]-min(frame)
            denom = max(frame)-min(frame)
            frame[x] = num/denom
        histogram.append(frame)
        limitator+=50
    return np.array(histogram, dtype=np.single)

In [7]:
def compute_descriptor(melody):
    cents = to_cents(melody)
    semitones = to_semitones(cents)
    mapped = map_into_single_octave(semitones)
    chroma = get_histogram(mapped)
    return chroma

<h1>=====================================</h1>

In [8]:
melodies_path = 'dataset/melodies/vocals_spleeter/'
#melodies_path = 'dataset/melodies/vocals_demucs/'
#melodies_path = 'dataset/melodies/full_songs/'
descriptors_path = 'dataset/chromas_sbbc/vocals/'
melodies = os.listdir(melodies_path)

In [9]:
for melody in melodies:
    pitch_values = read_melody_files(melodies_path + melody)
    chroma = compute_descriptor(pitch_values)
    np.savetxt(descriptors_path + melody.split('.')[0] + '.txt', chroma)